# Perform a differential gene expression analysis with brainscapes

The DifferentialGeneExpression class compute differential gene expresssion in two different brain regions, following the JuGEx algorithm described in the following publication:

> *Sebastian Bludau, Thomas W. Mühleisen, Simon B. Eickhoff, Michael J. Hawrylycz, Sven Cichon, Katrin Amunts. Integration of transcriptomic and cytoarchitectonic data implicates a role for MAOA and TAC1 in the limbic-cortical network. 2018, Brain Structure and Function. https://doi.org/10.1007/s00429-018-1620-6*
      
In brainscapes, the object is initialized with a brainscapes atlas object. It will check if the parcellation selected in the atlas is suitable for performing the analysis, which includes to verify that the given atlas object provides maps in the MNI ICBM 152 space. The analysis is configured by specifying some candidate genes of interest, and two regions of interest (ROI) specified by brain area names that the atlas object can resolve. Note that tha brainscapes atlas class does fuzzy string matching to resolve region names, so you can try with a simple name of the regions to see if brainscapes interprets them.  Also, gene names can easily be looked up and autocompleted in brainscapes.features.gene_names. 

For the gene expression data, brainscapes accesses the Allen Brain Atlas API (© 2015 Allen Institute for Brain Science. Allen Brain Atlas API. Available from: brain-map.org/api/index.html). 

## Preparation

We install the brainscapes library, and set an EBRAINS Knowledge Graph authentication token the we can request  [here](https://nexus-iam.humanbrainproject.org/v0/oauth2/authorize) after creating an HBP account.

In [ ]:
!pip install brainscapes
from os import environ
environ['HBP_AUTH_TOKEN'] = "<<<Paste your HBP auth token here>>>"

# Set cache dir. If unset, will use default user tempdir as returned by the appdirs module
# environ['BRAINSCAPES_CACHEDIR'] = "<<<Cache dir>>>"

## Instantiate human brain atlas with Julich-Brain cytoarchitectonic maps

We instantiate the human atlas, and tell it to prefer thresholded continous maps over labelled regions in the discrete parcellation map. For the Julich-Brain parcellation, this mean that it will use thresholded probability maps, if available.

In [ ]:
import brainscapes as bs
bs.logger.setLevel("INFO") # we want to see some messages!

In [ ]:
atlas = bs.atlases.MULTILEVEL_HUMAN_ATLAS

# next line is optional - cytoarchitectonic maps are selected by default
atlas.select_parcellation(bs.parcellations.JULICH_BRAIN_PROBABILISTIC_CYTOARCHITECTONIC_MAPS_V2_5_)

# as in the original JuGEx, we prefer thresholded probability maps 
# over the labelled region in the maximum probability map

threshold = $THRESHOLD$
atlas.enable_continuous_map_thresholding(threshold)

## Setup and run the experiment

In [ ]:
candidate_regions = [$AREA1$, $AREA2$]
candidate_genes = $GENELIST$
n_rep = $REPS$

Configuring the experiment is now just a matter of a few lines of code:

In [ ]:
jugex = bs.analysis.DifferentialGeneExpression(atlas)
jugex.add_candidate_genes(candidate_genes)
jugex.define_roi1(candidate_regions[0])
jugex.define_roi2(candidate_regions[1])

We are now setup to run the analysis.

In [ ]:
result = jugex.run(permutations=n_rep)

In [ ]:
print(result['p-values'])

## Look at the anatomical positions of the samples in MNI space

Let's have a look at the sample positions that have been found in the Allen atlas. Since we configured brainscapes to prefer thresholded continuous maps for region filtering over the simplified parcellation map, we also plot the probability maps here.

In [ ]:
from nilearn import plotting
import numpy as np

for region in candidate_regions:
    samples = jugex.get_samples(region)
    atlas.select_region(region)
    pmap = atlas.selected_region.get_specific_map(
        bs.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC)    
    # we could have also used the simple parcellation map mask as follows:
    # mask = atlas.get_mask(bs.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC)
    display = plotting.plot_roi(pmap,cmap='jet')
    display.add_markers([s['mnicoord'] for s in samples.values()])

The aggregated input parameters can be stored to disk.

In [ ]:
jugex.save('jugex_{}_{}.json'.format(
    "_".join(candidate_regions),
    "_".join(candidate_genes) ))